In [1]:
import music21
import pathlib
import subprocess

In [166]:
class ABCMusicConverter:
    score: music21.stream.base.Score | None
    destination: pathlib.Path
    filename: str
    midi_file: pathlib.Path | None
    wav_file: pathlib.Path | None
    mp3_file: pathlib.Path | None

    instruments: dict = {
        k.lower(): v
        for k, v in vars(music21.instrument).items()
        if hasattr(v, "bestName")
    }

    def __init__(self, abc: str, filename: str, destination: str | pathlib.Path = "."):
        self.destination = pathlib.Path(destination)
        self.filename = filename
        
        self.midi_file = None
        self.wav_file = None
        self.mp3_file = None

        self.score = music21.converter.parse(abc)

    def to_midi(
        self,
        midi_file: str | pathlib.Path | None = None,
        instrument: str | None = None,
        tempo: int | None = None,
    ) -> pathlib.Path:
        # Path to new midi file
        if midi_file is None:
            self.midi_file = (self.destination / self.filename).with_suffix(".mid")
        else:
            self.midi_file = pathlib.Path(midi_file)

        # Delete midi file if exists
        if self.midi_file.exists():
            self.midi_file.unlink()

        if instrument is not None:
            instrument = self.instruments.get(instrument.lower())
            for p in self.score.parts:
                p.insert(0, instrument())

        if tempo is not None:
            self.score.insert(0, music21.tempo.MetronomeMark(number=tempo))

        # Convert to midi
        mf = music21.midi.translate.music21ObjectToMidiFile(self.score)
        mf.open(self.midi_file, "wb")
        mf.write()
        mf.close()

        return self.midi_file

    def to_wav(
        self,
        wav_file: str | pathlib.Path | None = None,
        sound_font: str | pathlib.Path = "GeneralUser-GS.sf2",
        sampling_rate: int = 16000,
        **kwargs
    ) -> pathlib.Path:
        # Create midi file if necessary
        if self.midi_file is None:
            self.to_midi(**kwargs)

        # Path to new wav file
        if wav_file is None:
            self.wav_file = (self.destination / self.filename).with_suffix(".wav")
        else:
            self.wav_file = pathlib.Path(wav_file)

        # Remove file if exists
        if self.wav_file.exists():
            self.wav_file.unlink()

        # Check if sound_font exists
        sound_font = pathlib.Path(sound_font)
        assert sound_font.exists()

        # Convert to wav
        command = [
            "fluidsynth",
            "-ni",
            str(sound_font),
            str(self.midi_file),
            "-F",
            str(self.wav_file),
            "-r",
            str(sampling_rate),
        ]

        subprocess.run(command, check=True, capture_output=True)

        return self.wav_file

    def to_mp3(
        self, mp3_file: str | pathlib.Path | None = None, **kwargs
    ) -> pathlib.Path:
        # Create wave file if necessary
        if self.wav_file is None:
            self.to_wav(**kwargs)

        # Path to new mp3 file
        if mp3_file is None:
            self.mp3_file = (self.destination / self.filename).with_suffix(".mp3")
        else:
            self.mp3_file = pathlib.Path(mp3_file)

        # Remove file if exists
        if self.mp3_file.exists():
            self.mp3_file.unlink()

        command = ["ffmpeg", "-i", str(self.wav_file), str(self.mp3_file)]

        subprocess.run(command, check=True, capture_output=True)

        return self.mp3_file

In [167]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(instrument="violin", tempo=180)
ABCMusicConverter("cooleys.abc", "cooleys_flute").to_wav(instrument="flute", tempo=180)

PosixPath('cooleys_flute.wav')

# CLAP

In [4]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=False)  # disable fusion for audio-only use
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [5]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.wav")

In [6]:
import torchaudio

def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim


In [7]:
audio_tensor = load_audio("cooleys.wav")

In [8]:
with torch.no_grad():
    embedding = model.get_audio_embedding_from_data(audio_tensor, use_tensor=True)

In [9]:
embedding.shape

torch.Size([1, 512])

In [10]:
audio_tensor2 = load_audio("cooleys_flute.wav")
with torch.no_grad():
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)

In [11]:
embedding2

tensor([[-4.7669e-02,  1.5185e-02,  5.8923e-02, -1.2021e-02,  9.5482e-02,
         -3.4315e-03, -4.8566e-02,  1.8087e-02,  2.7128e-02,  1.4882e-02,
         -6.4931e-02, -5.3605e-03,  4.7362e-02, -1.5416e-03, -5.5228e-03,
         -4.7449e-02, -6.3107e-02, -2.0978e-02, -3.9053e-02,  7.6627e-02,
          2.2313e-02,  1.4599e-01, -1.2977e-02,  4.0569e-03,  1.7864e-02,
         -6.3704e-05,  1.7713e-02, -1.1361e-02,  1.8299e-02, -7.1580e-04,
          5.2289e-02,  1.0640e-01,  9.7967e-03, -7.6950e-02,  1.0185e-01,
         -6.2829e-02, -8.8202e-02, -1.9113e-02, -2.3685e-02,  6.1610e-02,
         -6.7963e-02,  1.8895e-02, -8.2265e-02,  6.9059e-03,  2.3369e-02,
         -5.4911e-02, -1.8842e-02,  9.8239e-03, -2.3676e-02,  3.2094e-03,
          2.8726e-02, -6.8386e-02, -2.8290e-02,  3.8179e-02,  2.3533e-02,
          2.7939e-02, -1.0502e-01, -2.0496e-02,  1.4185e-02,  2.3109e-02,
          1.3942e-03, -1.5327e-02,  2.7790e-02, -3.3877e-02,  7.1842e-03,
         -4.5241e-02,  1.3446e-02, -1.

In [12]:
torch.nn.functional.cosine_similarity(embedding, embedding2)

tensor([0.7682], device='cuda:0')

In [13]:
ABCMusicConverter("butterfly.abc").to_wav(instrument="violin", tempo=160)

PosixPath('butterfly.wav')

In [14]:
ABCMusicConverter("butterfly.abc").to_wav("butterfly_flute.wav", instrument="flute", tempo=160)

PosixPath('butterfly_flute.wav')

In [15]:
audio_tensor3 = load_audio("butterfly.wav")
with torch.no_grad():
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

In [16]:
audio_tensor4 = load_audio("butterfly_flute.wav")
with torch.no_grad():
    embedding4 = model.get_audio_embedding_from_data(audio_tensor4, use_tensor=True)

In [17]:
torch.nn.functional.cosine_similarity(embedding3, embedding4)

tensor([0.8442], device='cuda:0')

In [18]:
import torch
import torch.nn.functional as F

def nt_xent_loss(z1, z2, temperature=0.07):
    """
    Contrastive loss using implicit negatives (NT-Xent).
    Args:
        z1: Tensor of shape (N, D) – embeddings from view 1 (e.g., anchors)
        z2: Tensor of shape (N, D) – embeddings from view 2 (e.g., positives)
    Returns:
        Scalar contrastive loss
    """
    batch_size = z1.size(0)

    # Normalize embeddings
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    # Concatenate for full 2N x D
    z = torch.cat([z1, z2], dim=0)  # shape: (2N, D)

    # Cosine similarity matrix (2N x 2N)
    sim = torch.matmul(z, z.T) / temperature  # shape: (2N, 2N)

    # Mask self-similarity
    mask = torch.eye(2 * batch_size, device=z.device).bool()
    sim.masked_fill_(mask, -float('inf'))  # ignore similarity to self

    # Targets: for i in 0..N-1, positive pair is i<->i+N and i+N<->i
    targets = torch.cat([
        torch.arange(batch_size, 2 * batch_size),
        torch.arange(0, batch_size)
    ]).to(z.device)

    loss = F.cross_entropy(sim, targets)
    return loss

In [19]:
z1 = torch.cat([embedding, embedding3], dim=0)

In [20]:
z2 = torch.cat([embedding2, embedding4], dim=0)

In [21]:
nt_xent_loss(z1, z2)

tensor(2.1133, device='cuda:0')

In [22]:
from bs4 import BeautifulSoup

def extract_abc_notation(html: str, abc_id: str = "abc1") -> str:
    """
    Extracts ABC music notation from a specific <div> block in given HTML.

    Parameters:
        html (str): The full HTML content as a string.
        abc_id (str): The ID of the ABC notation block to extract (default: "abc7").

    Returns:
        str: The extracted ABC notation as a plain text string.
    """
    soup = BeautifulSoup(html, 'html.parser')

    abc_div = soup.find("div", {"class": "setting-abc", "id": abc_id})
    if not abc_div:
        raise ValueError(f"No div found with id '{abc_id}'")

    notes_div = abc_div.find("div", {"class": f"notes"})
    if not notes_div:
        raise ValueError(f"No notes div found")

    # Extract the text, convert HTML entities, and strip leading/trailing whitespace
    abc_text = notes_div.get_text()
    return abc_text.strip()


# SCRAPER

In [1]:
import asyncio
import inspect
import numpy as np
from playwright.async_api import async_playwright

class Scraper:
    def __init__(self, crawl_delay=(11, 2), headless=True, max_concurrency=3):
        self.crawl_delay = crawl_delay
        self.headless = headless
        self.browser = None
        self.context = None

        # Setting the number of parallel requests
        self.max_concurrency = max_concurrency
        self.semaphore = asyncio.Semaphore(max_concurrency)

    async def start(self):
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.chromium.launch(headless=self.headless)
        self.context = await self.browser.new_context(user_agent=(
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        ))

    async def close(self):
        if self.context:
            await self.context.close()
        if self.browser:
            await self.browser.close()
        if self.playwright:
            await self.playwright.stop()

    async def sleep_crawl_delay(self):
        # Wait a duration distributed normally around mean with standard deviation std
        await asyncio.sleep(np.random.normal(*self.crawl_delay))

    async def fetch_page(self, url: str, on_result=None, **kwargs) -> str:
        await self.sleep_crawl_delay()
        page = await self.context.new_page()
        await page.goto(url)
        html = await page.content()
        await page.close()

        if on_result:
            if inspect.iscoroutinefunction(on_result):
                result = await on_result(url, html, **kwargs)
                return result
            else:
                return on_result(url, html, **kwargs)
        else:
            return html
    
    async def fetch_pages(self, urls: list[str], on_result=None, **kwargs) -> dict:
        # Schedule fetches concurrently but limited by semaphore
        tasks = [self.fetch_page(url, on_result, **kwargs) for url in urls]
        results = await asyncio.gather(*tasks)
        return dict(zip(urls, results))
    
    async def __aenter__(self):
        await self.start()
        return self

    async def __aexit__(self, *exc):
        await self.close()


In [2]:
def extract_abc_versions(html):
    i = 1
    versions = []

    while True:
        try:
            abc = extract_abc_notation(html, abc_id=f"abc{i}")
            versions.append(abc)
            i += 1
        except ValueError:
            break
    
    return versions


In [ ]:
def extract_title_and_aliases(html):
    soup = BeautifulSoup(html, "html.parser")

    # Extract main title from <h1>
    h1_tag = soup.find("h1")
    title = h1_tag.contents[0].strip() if h1_tag else None

    # Extract "Also known as" names from <p class="info">
    info_p = soup.find("p", class_="info")
    aliases = []
    if info_p and info_p.text.startswith("Also known as"):
        raw = info_p.text.replace("Also known as", "").strip()
        aliases = [name.strip() for name in raw.split(",")]

    return {
        "title": title,
        "aliases": aliases
    }


In [ ]:
import re

def sanitize_title(title):
    # Replace spaces with underscores
    title = title.replace(" ", "_")
    # Remove any character that is NOT alphanumeric, underscore, hyphen, or dot
    title = re.sub(r"[^A-Za-z0-9_\-\.]", "", title)
    # Optionally, truncate length to e.g. 100 chars
    return title[:100].lower()

In [128]:
def parse_page(url, html, root_dir=".", database="database.db"):
    # Fetch title, aliases and ABC versions
    results = extract_title_and_aliases(html)
    results["url"] = url
    results["versions"] = extract_abc_versions(html)
    
    # Create destination directory
    root_dir = pathlib.Path(root_dir)
    root_dir.mkdir(exist_ok=True)
    
    number = url.lstrip("https://thesession.org/tunes")
    results["number"] = int(number)
    title = sanitize_title(results["title"])
    dest = root_dir / f"{number}_{title}"
    dest.mkdir(exist_ok=True)

    # write files
    for i, v in enumerate(results["versions"]):
        with open(dest / f"{number}_{title}_{i+1}.abc", "w", encoding="utf-8") as f:
            f.write(v)

    return results
    

In [68]:
urls = [
    "https://thesession.org/tunes/1",
    "https://thesession.org/tunes/2",
    "https://thesession.org/tunes/3",
]

async with Scraper(crawl_delay=(10,2), max_concurrency=2) as scraper:
    results = await scraper.fetch_pages(urls)

In [147]:
import sqlite3
database = pathlib.Path("database.db")

if database.exists():
    database.unlink()

with sqlite3.connect(database) as con:
    # Create schema
    with open("database.sql", "r") as f:
        con.executescript(f.read())

con.close()

In [148]:
import asyncio
import aiosqlite

class AsyncSQLiteWriter:
    def __init__(self, db_path):
        self.db_path = db_path
        self.queue = asyncio.Queue()
        self.task = None

    async def start(self):
        self.task = asyncio.create_task(self.worker())

    async def stop(self):
        await self.queue.put(None)  # Sentinel to stop worker
        await self.task

    async def worker(self):
        print("Starting worker...")
        async with aiosqlite.connect(self.db_path) as db:           
            while True:
                item = await self.queue.get()
                if item is None:
                    break
                
                tune_id = item["number"]
                print(tune_id, item["title"])

                await db.execute(
                    "INSERT OR REPLACE INTO tunes (TuneID, TuneTitle, TuneURL) VALUES (?, ?, ?)",
                    (tune_id, item["title"], item["url"])
                )

                await db.execute("DELETE FROM TuneAliases WHERE TuneID = ?", (tune_id,))
                await db.execute("DELETE FROM TuneVersions WHERE TuneID = ?", (tune_id,))

                await db.executemany(
                    "INSERT INTO TuneAliases (TuneID, TuneAlias) VALUES (?, ?)",
                    [(tune_id, a) for a in item["aliases"]]
                )

                await db.executemany(
                    "INSERT INTO TuneVersions (TuneID, TuneVersion) VALUES (?, ?)",
                    [(tune_id, v) for v in item["versions"]]
                )

                await db.commit()

    async def write(self, results):
        await self.queue.put(results)

    async def __aenter__(self):
        await self.start()
        return self

    async def __aexit__(self, *exc):
        await self.stop()
        


In [149]:
async def parse_page(url, html, sqlite_writer=None):
    # Fetch title, aliases and ABC versions
    results = extract_title_and_aliases(html)
    results["url"] = url
    results["versions"] = extract_abc_versions(html)
    results["number"] = int(url.lstrip("https://thesession.org/tunes/"))
    
    if sqlite_writer is not None:
        await sqlite_writer.write(results)

    return results
    

In [150]:
urls = [f"https://thesession.org/tunes/{i}" for i in range(500)]

async with AsyncSQLiteWriter("database.db") as writer:
    async with Scraper(crawl_delay=(60,10), max_concurrency=1) as scraper:
        results = await scraper.fetch_pages(urls, on_result=parse_page, sqlite_writer=writer)

Starting worker...
265 The Cobbler’s
2 The Bucks Of Oranmore
214 McDonagh’s
271 Brian Boru’s March
411 The Heather Breeze
365 The Cat Rambles To The Child’s Saucepan
8 The Banshee
73 The Musical Priest
460 Larry Grogan
346 Music For A Found Harmonium
300 The Dreary Plains Of Toil
162 Bill Malley’s
390 The Famous Ballymote
58 The Lads Of Laois
164 The Hen And Her Brood
193 The Windmill
268 Do It Fair
434 Christmas In Killarney
166 The Green Mountain
484 The House On The Hill
157 Cathleen Hehir’s
386 The Galtee Hunt
29 Dusty Windowsills
55 The Kesh
22 The Dancing Dog
17 The Coleraine
311 Colonel Rodgers’ Favourite
226 The Convenience
23 The Dingle Regatta
141 The Humours Of Tulla
422 A Parcel Of Land
250 The Road To Lisdoonvarna
375 Sandy MacIntyre’s Trip To Boston
427 When Sick Is It Tea That You Want?
282 Joel And Kelly’s
215 Mairtin O’Connor’s Flying Clog
473 Bruno
52 The Kid On The Mountain
142 The Boyne Hunt
33 Farewell To Ireland
39 The Kerry
113 Toss The Feathers
168 The Nova Scot

In [151]:
urls = [f"https://thesession.org/tunes/{i}" for i in range(500, 10000)]

async with AsyncSQLiteWriter("database.db") as writer:
    async with Scraper(crawl_delay=(90,10), max_concurrency=1) as scraper:
        results = await scraper.fetch_pages(urls, on_result=parse_page, sqlite_writer=writer)

Starting worker...
3188 The Solstice
3422 Carolan’s Cup
7299 410
9341 In Memory Of Chris Langan
6550 The Floating Carpet
8207 410
7392 The Auld Fiddler
2130 John Kelly’s
7850 Snow Leopard
1433 Tommy Mulhaire’s
8392 Reel D’Espagne
4744 The Antigonish Polka
2078 The Horse In The Houseboat
4072 The Glen Aln
5457 Liam’s Favourite Book
5027 Eddie Moloney’s
3807 The Fairies’ Revels
5201 410
2255 Gan Ainm
6689 Pencarrow
8341 Lady’s Plaything
9755 The Keel Row
6409 Paddy Fahey’s
4039 The Thrush In The Bush
4663 Sandy M’Gaff
6596 Balindore
2107 Bulgarian Red
7808 410
9425 Dornoch Links
2893 410
7391 The Horizontal
4475 The First Day Of Spring
2774 Willy’s Single
764 410
6985 Redford Cottage
3372 410
6595 Morris Off
5906 The Atlanta Schottische


TargetClosedError: BrowserContext.close: Target page, context or browser has been closed
Browser logs:

<launching> /home/maxime/.cache/ms-playwright/chromium_headless_shell-1169/chrome-linux/headless_shell --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AcceptCHFrame,AutoExpandDetailsElement,AvoidUnnecessaryBeforeUnloadCheckSync,CertificateTransparencyComponentUpdater,DeferRendererTasksAfterInput,DestroyProfileOnBrowserClose,DialMediaRouteProvider,ExtensionManifestV2Disabled,GlobalMediaControls,HttpsUpgrades,ImprovedCookieControls,LazyFrameLoading,LensOverlay,MediaRouter,PaintHolding,ThirdPartyStoragePartitioning,Tra

In [153]:
import pandas as pd

with sqlite3.connect("database.db") as con:
    df = pd.read_sql("SELECT TuneVersion FROM TuneVersions JOIN Tunes USING (TuneID) WHERE TuneTitle LIKE '%Cooley%'", con)

In [174]:
print(df.loc[13, "TuneVersion"])

X: 14
T: Cooley's
R: reel
M: 4/4
L: 1/8
K: Edor
|:D2|EB{c}BA B2 EB|~B2 AB dBAG|FDAD BDAD|FDAD dAFD|
EBBA B2 EB|B2 AB defg|afe^c dBAF|DEFD E2:|
|:gf|eB B2 efge|eB B2 gedB|A2 FA DAFA|A2 FA defg|
eB B2 eBgB|eB B2 defg|afe^c dBAF|DEFD E2:|


In [158]:
score = music21.converter.parse(df.loc[1, "TuneVersion"])

In [172]:
ABCMusicConverter(df.loc[13, "TuneVersion"], "cooleys2").to_wav(instrument="violin", tempo=160)

PosixPath('cooleys2.wav')

In [175]:
con.close()